In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.agents import tool
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
import textwrap
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.tools import Tool
import langchain
from slack_sdk import WebClient
import json
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain.schema import Document
from typing import Iterator, List, Literal, Optional, Sequence, Union
from langchain.document_loaders.blob_loaders import FileSystemBlobLoader
from langchain.document_loaders.blob_loaders import Blob
import os
import openai

In [17]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']


In [4]:
pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.1/421.1 kB 11.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.2 MB/s eta 0:00:0000:01

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pydub


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
class OpenAIWhisperParserCustom(OpenAIWhisperParser):
    """Transcribe and parse audio files.
    Audio transcription is with OpenAI Whisper model."""

    def __init__(self, api_key: Optional[str] = None,chunk_duration=10):
        self.api_key = api_key
        self.chunk_duration = chunk_duration # in  seconds

    def lazy_parse(self, blob: Blob) -> Iterator[Document]:
        """Lazily parse the blob."""

        import io

        try:
            import openai
        except ImportError:
            raise ValueError(
                "openai package not found, please install it with "
                "`pip install openai`"
            )
        try:
            from pydub import AudioSegment
        except ImportError:
            raise ValueError(
                "pydub package not found, please install it with " "`pip install pydub`"
            )

        # Set the API key if provided
        if self.api_key:
            openai.api_key = self.api_key

        # Audio file from disk
        audio = AudioSegment.from_file(blob.path)

        # Chunk duration is set to 10 secs for the purpose of time correlation
        chunk_duration_ms = self.chunk_duration * 1000

        # Split the audio into chunk_duration_ms chunks
        for split_number, i in enumerate(range(0, len(audio), chunk_duration_ms)):
            # Audio chunk
            chunk = audio[i : i + chunk_duration_ms]
            file_obj = io.BytesIO(chunk.export(format="mp3").read())
            if blob.source is not None:
                file_obj.name = blob.source + f"_part_{split_number}.mp3"
            else:
                file_obj.name = f"part_{split_number}.mp3"

            # Transcribe
            print(f"Transcribing part {split_number+1}!")
            #transcript = "hello"#openai.Audio.transcribe("whisper-1", file_obj)
            transcript = openai.Audio.transcribe("whisper-1", file_obj)

            yield Document(
                page_content=transcript.text,
                #page_content=transcript,
                metadata={"source": blob.source, "chunk": split_number, "from": i, "to":i+chunk_duration_ms},
            )


In [4]:
from typing import Iterable, List

from langchain.document_loaders.blob_loaders import FileSystemBlobLoader
from langchain.document_loaders.blob_loaders.schema import Blob, BlobLoader

class YoutubeAudioLoaderCustom(BlobLoader):

    """Load YouTube urls as audio file(s)."""

    def __init__(self, urls: List[str], save_dir: str):
        if not isinstance(urls, list):
            raise TypeError("urls must be a list")

        self.urls = urls
        self.save_dir = save_dir

    def yield_blobs(self) -> Iterable[Blob]:
        """Yield audio blobs for each url."""

        try:
            import yt_dlp
        except ImportError:
            raise ValueError(
                "yt_dlp package not found, please install it with "
                "`pip install yt_dlp`"
            )

        # Use yt_dlp to download audio given a YouTube url
        ydl_opts = {
            "format": "m4a/bestaudio/best",
            "noplaylist": True,
            "outtmpl": self.save_dir + "/%(title)s.%(ext)s",
            "postprocessors": [
                {
                    "key": "FFmpegExtractAudio",
                    "preferredcodec": "m4a",
                    
                }
            ],
            "ffmpeg_location":"/Users/hbolak650/Downloads"
        }

        for url in self.urls:
            # Download file
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download(url)

        # Yield the written blobs
        loader = FileSystemBlobLoader(self.save_dir, glob="*.m4a")
        for blob in loader.yield_blobs():
            yield blob


In [5]:
!pip install ffprobe

  Preparing metadata (setup.py) ... done
  Created wheel for ffprobe: filename=ffprobe-0.5-py3-none-any.whl size=3407 sha256=6c7c2e091046ae365b4c771f44daed88e39b05e6c323c762230607a1c1975667
  Stored in directory: /Users/hbolak650/Library/Caches/pip/wheels/3e/d8/a3/2ee15021cfa0b180fcbc0864f71047237a04dcb3933a3d698f
Successfully built ffprobe

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [6]:

os.environ["LANGCHAIN_DEBUG"] = "true"
#  Karpathy lecture videos
urls = ["https://www.youtube.com/watch?v=_xASV0YmROc"]


# Directory to save audio files
save_dir = "/Users/hbolak650/Downloads/YouTubeYoga"

# Transcribe the videos to text
loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParserCustom())
try:
    docs = loader.load()
    if len(docs) == 0:
        print("No documents loaded.")
    else:
        for doc in docs:
            # Process the loaded documents
            print(doc)
except Exception as e:
    print("Error loading documents:", str(e))    

os.environ["LANGCHAIN_DEBUG"] = "false"   

[youtube] Extracting URL: https://www.youtube.com/watch?v=_xASV0YmROc
[youtube] _xASV0YmROc: Downloading webpage
[youtube] _xASV0YmROc: Downloading ios player API JSON
[youtube] _xASV0YmROc: Downloading android player API JSON
[youtube] _xASV0YmROc: Downloading m3u8 information
[info] _xASV0YmROc: Downloading 1 format(s): 140
[download] /Users/hbolak650/Downloads/YouTubeYoga/The 3-Minute Yoga Routine To Jump Start Your Mornings.m4a has already been downloaded
[download] 100% of    2.70MiB
[ExtractAudio] Not converting audio /Users/hbolak650/Downloads/YouTubeYoga/The 3-Minute Yoga Routine To Jump Start Your Mornings.m4a; file is already in target format m4a
Transcribing part 1!
Transcribing part 2!
Transcribing part 3!
Transcribing part 4!
Transcribing part 5!
Transcribing part 6!
Transcribing part 7!
Transcribing part 8!
Transcribing part 9!
Transcribing part 10!
Transcribing part 11!
Transcribing part 12!
Transcribing part 13!
Transcribing part 14!
Transcribing part 15!
Transcribing p

In [7]:
# Returns a list of Documents, which can be easily viewed or parsed
len(docs)

18

In [8]:
print(docs[0])

page_content="We're going to take a deep breath in, on the exhale, we're going to lift all the way up, airplane down and taking your head towards the sky." metadata={'source': '/Users/hbolak650/Downloads/YouTubeYoga/The 3-Minute Yoga Routine To Jump Start Your Mornings.m4a', 'chunk': 0, 'from': 0, 'to': 10000}


In [10]:
# Combine doc
combined_docs = [doc.page_content for doc in docs]
text = " ".join(combined_docs)
text

"We're going to take a deep breath in, on the exhale, we're going to lift all the way up, airplane down and taking your head towards the sky. your knee. From here, I want you to look out as if you're looking onto the horizon and then you're going to slowly come back. Let's try that again. So we lift, look onto the horizon. From here, we're going to slowly take your right leg back into a lunge, and you're in this position nice and straight. Think of lengthening out, you're going to stretch out through that hamstring. From here, we're going to lengthen the hips up, and toes come up. Take your head down to the shin. round over just like a cat now we're going to try and rotate towards that front knee rotate so you're going to feel that openness in your back Open up through your chest and you're pressing your hips down to the floor. Remember, breathe in that pose, taking deep breaths, come back to center. Now we're going to pass through into a plank. to a plank position. You're pushing away

In [11]:
#embeddings = OpenAIEmbeddings()
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

/Users/hbolak650/.local/share/virtualenvs/SoftwareStudio-BV5WFfKi/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
PERSIST_DIRECTORY_OPENAI = "../tubegptdb"
 
tubegptdb = Chroma.from_documents(
    documents=docs, embedding= embeddings, persist_directory=PERSIST_DIRECTORY_OPENAI
)
tubegptdb.similarity_search_with_score("What was the yoga move?", k=2)

[(Document(page_content='bend your knees, rolling through the spine exhale, bring the legs together and down, one more time', metadata={'source': '/Users/hbolak650/Downloads/YouTubeYoga/The 3-Minute Yoga Routine To Jump Start Your Mornings.m4a', 'chunk': 15, 'from': 150000, 'to': 160000}),
  0.9614589214324951),
 (Document(page_content="Open up through your chest and you're pressing your hips down to the floor. Remember, breathe in that pose, taking deep breaths, come back to center. Now we're going to pass through into a plank.", metadata={'source': '/Users/hbolak650/Downloads/YouTubeYoga/The 3-Minute Yoga Routine To Jump Start Your Mornings.m4a', 'chunk': 5, 'from': 50000, 'to': 60000}),
  0.984209418296814)]

In [13]:
tubegptdb.persist()

In [14]:
template = """You're a Professor who helps students on Youtube contents .
 
{context}
 
Answer with Accurate data   to the question and the way Professor speaks. 
 
Question: {question}
Answer:"""
 
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
print(
    prompt.format(
        context="A youtube user asking about contents ",
        question="Why do we need to zero out the gradient before backprop at each step?",
    )
)

You're a Professor who helps students on Youtube contents .
 
A youtube user asking about contents 
 
Answer with Accurate data   to the question and the way Professor speaks. 
 
Question: Why do we need to zero out the gradient before backprop at each step?
Answer:


In [15]:
chain_type_kwargs = {"prompt": prompt}
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=tubegptdb.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

In [16]:
query = "What was the move after looking into Horizon"
response = chain.run(query)
print_response(response)

NameError: name 'print_response' is not defined

In [18]:
print_response(response)

After looking onto the horizon, the next move is to slowly come back and return to the original
position.
